In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

from collections import Counter
import operator

In [2]:
filename = '../data/2019-Oct.csv_10%.csv'

In [3]:
import random
p = 0.5  # 1% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
df = pd.read_csv(
         filename,
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)
df.shape

(2124092, 9)

# Preprocess

In [4]:
def preprocessing_feat(X, drop_event_time=False):
    if drop_event_time:
        X = X.drop("event_time", axis=1)
    
    X_preprocessed = X.dropna(subset = ['category_code', 'brand']) #tbd!!
    X_preprocessed = X_preprocessed.drop_duplicates()
    X_preprocessed['category_code'] = X_preprocessed['category_code'].str.replace('.',' ')
    return X_preprocessed

In [5]:
X_preprocessed = preprocessing_feat(df, drop_event_time=True)

/tmp/ipykernel_20101/4055017079.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X_preprocessed['category_code'] = X_preprocessed['category_code'].str.replace('.',' ')


In [6]:
X_preprocessed.shape

(1292625, 8)

In [7]:
def make_column(row):
    if row["price"] < row["25%"]:
        return "low"
    elif row["price"] < row["75%"]:
        return "medium"
    else:
        return "high"

def pricing_criterion(X):
    pricing_guide = X.groupby('category_code')['price'].describe()[["25%", "75%"]].reset_index()
    X_merged = X.merge(pricing_guide, on="category_code", how="right")
    X_merged["price_category"] = X_merged.apply(lambda row: make_column(row), axis=1)
    return X_merged


In [8]:
X_merged = pricing_criterion(X_preprocessed)

In [9]:
def metadata(X):
    X['metadata'] = X[['category_code', 'brand', 'price_category']].apply(lambda x: ' '.join(x), axis = 1)
    return X


In [10]:
X_meta = metadata(X_merged)
X_meta.shape

(1292625, 12)

In [11]:
X_meta.head()

,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,25%,75%,price_category,metadata
0,view,32900083,2055156924407612189,accessories bag,a-elita,8.49,549437633,62760980-f427-42e0-a891-8b590fd54584,16.28,76.945,low,accessories bag a-elita low
1,view,28401176,2053013566209917945,accessories bag,karya,100.39,549348524,933c58ab-5185-44d6-94d6-c2ccb335d7f9,16.28,76.945,high,accessories bag karya high
2,view,32900008,2055156924407612189,accessories bag,shimano,95.21,514511864,557ce000-26cc-412c-a2be-3830a0b455c3,16.28,76.945,high,accessories bag shimano high
3,view,28401054,2053013566209917945,accessories bag,karya,100.39,514312434,0d0c3d87-ed6f-4f58-9b9e-2cc44b89d25c,16.28,76.945,high,accessories bag karya high
4,view,32900106,2055156924407612189,accessories bag,prologic,17.68,532998360,e3fc00a8-70cf-4acb-b3df-66f10d461f7b,16.28,76.945,medium,accessories bag prologic medium


In [12]:
X_purch = X_meta[X_meta['event_type']=='purchase']

In [13]:
X_purch.head()

,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,25%,75%,price_category,metadata
120,purchase,49800012,2126679654801604876,accessories bag,ritmix,9.77,518567988,d577a234-832c-4772-84da-5bfcce6b5951,16.28,76.945,low,accessories bag ritmix low
138,purchase,18300207,2053013558945383017,accessories bag,continent,25.71,548244649,8f336749-d546-4489-9649-7c8f7d970d2a,16.28,76.945,medium,accessories bag continent medium
355,purchase,28400775,2053013566209917945,accessories bag,baden,61.52,520845679,48c5985f-7cb2-4ec9-9b7f-2410d9afc55c,16.28,76.945,medium,accessories bag baden medium
545,purchase,49800017,2126679654801604876,accessories bag,ritmix,12.32,512700216,d1faefa6-ce4a-474e-b952-26b0483a6cfa,16.28,76.945,low,accessories bag ritmix low
594,purchase,28400343,2053013566209917945,accessories bag,karya,100.39,522957356,29d61a4a-c1fa-4c38-acd0-8685cac830eb,16.28,76.945,high,accessories bag karya high


In [14]:
def filter(X):
    df = pd.DataFrame(X.groupby('user_session').event_type.count())
    df_new = df[df['event_type']>1]
    return df_new

In [15]:
X_filter = filter(X_purch)

In [16]:
df_cross = X_purch[X_purch['user_session'].isin(X_filter.index)]

In [17]:
df_cross.head()

,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,25%,75%,price_category,metadata
30374,purchase,28712682,2053013565228450757,apparel shoes,marcomen,75.42,551848382,ab0cd54c-7934-47b7-8c68-75d86482badc,62.81,105.54,medium,apparel shoes marcomen medium
37221,purchase,28716499,2053013565782098913,apparel shoes,respect,154.19,526900210,95e878f9-c7cf-437b-9d3e-3463cb915cf6,62.81,105.54,high,apparel shoes respect high
37225,purchase,28712327,2053013565639492569,apparel shoes,baden,64.61,526900210,95e878f9-c7cf-437b-9d3e-3463cb915cf6,62.81,105.54,medium,apparel shoes baden medium
54539,purchase,28715756,2053013565069067197,apparel shoes keds,nexpero,75.42,551848382,ab0cd54c-7934-47b7-8c68-75d86482badc,51.22,100.72,medium,apparel shoes keds nexpero medium
58072,purchase,28718443,2053013565069067197,apparel shoes keds,strobbs,51.22,560882745,b68ebe53-f014-4335-9ee9-abf31b749a34,51.22,100.72,medium,apparel shoes keds strobbs medium


# Counter

In [18]:
df_test = pd.DataFrame(df_cross.groupby('user_session')['product_id'].apply(list).tolist())
df_test.head()

,0,1,2
0,5100722,1004767,NaN
1,1004258,1002544,NaN
2,1004809,1004258,NaN
3,1005160,11300053,NaN
4,1003317,1002524,NaN


In [19]:
def concat(X):
    X = X[X[0]!=X[1]]
    X['conc'] = X[0].astype(str) + '_' + X[1].astype(str)
    
    lst = list(X['conc'])
    
    a = list(Counter(lst).keys()) # equals to list(X['conc'])
    b = list(Counter(lst).values()) # counts the elements' frequency
    
    dct_1 = {a[i]: b[i] for i in range(len(a))}
    dct_2 = dict(sorted(dct_1.items(), key=operator.itemgetter(1),reverse=True))

    return pd.DataFrame.from_dict(dct_2, orient='index').reset_index().rename(columns={'index': 'combo', 0: 'counts'})


In [20]:
X_sell = concat(df_test)

/tmp/ipykernel_20101/1906696682.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['conc'] = X[0].astype(str) + '_' + X[1].astype(str)


In [21]:
X_sell

,combo,counts
0,1005115_1005105,2
1,1004767_1004836,2
2,5100722_1004767,1
3,1004258_1002544,1
4,1004809_1004258,1
...,...,...
131,1005104_1003310,1
132,1005115_1004258,1
133,1004768_1004767,1
134,1004781_1004565,1


In [22]:
def x_seller(X):
    X["product_1"] = X["combo"].str.split("_",expand=True)[0].astype('category')
    X["product_2"] = X["combo"].str.split("_",expand=True)[1].astype('category')
    return X
    

In [23]:
new_dataframe = x_seller(X_sell)

new_dataframe.product_1 = new_dataframe.product_1.apply(lambda x: int(x))
new_dataframe.product_2 = new_dataframe.product_2.apply(lambda x: int(x))


In [24]:
new_df_1 = new_dataframe.merge(X_meta, how='left', left_on='product_1', right_on='product_id').rename(columns={'price': 'price_1', 'metadata': 'metadata_1'})

In [25]:
new_df_1.head()

,combo,counts,product_1,product_2,event_type,product_id,category_id,category_code,brand,price_1,user_id,user_session,25%,75%,price_category,metadata_1
0,1005115_1005105,2,1005115,1005105,view,1005115,2053013555631882655,electronics smartphone,apple,975.57,534574204,961ff93c-0af2-4319-9535-32f6f3fc08fe,179.26,725.08,high,electronics smartphone apple high
1,1005115_1005105,2,1005115,1005105,view,1005115,2053013555631882655,electronics smartphone,apple,975.57,548691513,c4439a54-98b1-4e8f-9d0e-f22ce6b1dea5,179.26,725.08,high,electronics smartphone apple high
2,1005115_1005105,2,1005115,1005105,view,1005115,2053013555631882655,electronics smartphone,apple,975.57,550422448,f9e45568-7f95-4367-8383-7dfdbd5c1791,179.26,725.08,high,electronics smartphone apple high
3,1005115_1005105,2,1005115,1005105,view,1005115,2053013555631882655,electronics smartphone,apple,975.57,518840573,929cf727-5d75-4eeb-b3d5-0bd631e57a11,179.26,725.08,high,electronics smartphone apple high
4,1005115_1005105,2,1005115,1005105,view,1005115,2053013555631882655,electronics smartphone,apple,975.57,525234636,8e8e11f1-8f68-405b-8cfa-ffb67fff78d2,179.26,725.08,high,electronics smartphone apple high


In [26]:
new_df_1 = new_df_1.drop_duplicates(['product_1','product_2'])
new_df_1.shape

(136, 16)

In [27]:
new_df_1.head()

,combo,counts,product_1,product_2,event_type,product_id,category_id,category_code,brand,price_1,user_id,user_session,25%,75%,price_category,metadata_1
0,1005115_1005105,2,1005115,1005105,view,1005115,2053013555631882655,electronics smartphone,apple,975.57,534574204,961ff93c-0af2-4319-9535-32f6f3fc08fe,179.26,725.08,high,electronics smartphone apple high
17059,1004767_1004836,2,1004767,1004836,view,1004767,2053013555631882655,electronics smartphone,samsung,254.82,555448677,3886fcc8-3618-4bb0-a9f4-614bfb2d5263,179.26,725.08,medium,electronics smartphone samsung medium
38487,5100722_1004767,1,5100722,1004767,view,5100722,2053013553341792533,electronics clocks,huawei,169.63,534060198,19071ee1-3abe-445e-b8b3-626f287aad81,71.53,379.42,medium,electronics clocks huawei medium
38933,1004258_1002544,1,1004258,1002544,view,1004258,2053013555631882655,electronics smartphone,apple,735.05,527049978,62164f60-662f-4a76-8bc8-234a1972910c,179.26,725.08,high,electronics smartphone apple high
43331,1004809_1004258,1,1004809,1004258,view,1004809,2053013555631882655,electronics smartphone,xiaomi,125.58,512681476,06a9e6ae-27ee-45a1-8c79-4693cfff0a21,179.26,725.08,low,electronics smartphone xiaomi low


In [28]:
df_short = new_df_1[['combo', 'counts', 'product_1', 'product_2', 'price_1', 'metadata_1']]
df_short.head()

,combo,counts,product_1,product_2,price_1,metadata_1
0,1005115_1005105,2,1005115,1005105,975.57,electronics smartphone apple high
17059,1004767_1004836,2,1004767,1004836,254.82,electronics smartphone samsung medium
38487,5100722_1004767,1,5100722,1004767,169.63,electronics clocks huawei medium
38933,1004258_1002544,1,1004258,1002544,735.05,electronics smartphone apple high
43331,1004809_1004258,1,1004809,1004258,125.58,electronics smartphone xiaomi low


In [29]:
df_final = df_short.merge(X_meta, how='left', left_on='product_2', right_on='product_id').rename(columns={'price': 'price_2', 'metadata': 'metadata_2'}).drop_duplicates(['product_1','product_2'])

In [30]:
df_final

,combo,counts,product_1,product_2,price_1,metadata_1,event_type,product_id,category_id,category_code,brand,price_2,user_id,user_session,25%,75%,price_category,metadata_2
0,1005115_1005105,2,1005115,1005105,975.57,electronics smartphone apple high,view,1005105,2053013555631882655,electronics smartphone,apple,1415.48,554832778,e71b31e8-623c-430b-a5b4-4c3089a77195,179.26,725.08,high,electronics smartphone apple high
10512,1004767_1004836,2,1004767,1004836,254.82,electronics smartphone samsung medium,view,1004836,2053013555631882655,electronics smartphone,samsung,241.18,524173350,5dfbfb82-0d2f-4d39-b7fd-9e3d1b2cd2fe,179.26,725.08,medium,electronics smartphone samsung medium
18575,5100722_1004767,1,5100722,1004767,169.63,electronics clocks huawei medium,view,1004767,2053013555631882655,electronics smartphone,samsung,254.82,555448677,3886fcc8-3618-4bb0-a9f4-614bfb2d5263,179.26,725.08,medium,electronics smartphone samsung medium
40003,1004258_1002544,1,1004258,1002544,735.05,electronics smartphone apple high,view,1002544,2053013555631882655,electronics smartphone,apple,464.13,515544667,f619453a-5696-43ba-8cd5-3752e31d811d,179.26,725.08,medium,electronics smartphone apple medium
49982,1004809_1004258,1,1004809,1004258,125.58,electronics smartphone xiaomi low,view,1004258,2053013555631882655,electronics smartphone,apple,735.05,527049978,62164f60-662f-4a76-8bc8-234a1972910c,179.26,725.08,high,electronics smartphone apple high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743425,1005104_1003310,1,1005104,1003310,975.57,electronics smartphone apple high,view,1003310,2053013555631882655,electronics smartphone,apple,747.23,555464729,504001a6-7129-4efc-bfdc-4edec6902f82,179.26,725.08,high,electronics smartphone apple high
746146,1005115_1004258,1,1005115,1004258,975.57,electronics smartphone apple high,view,1004258,2053013555631882655,electronics smartphone,apple,735.05,527049978,62164f60-662f-4a76-8bc8-234a1972910c,179.26,725.08,high,electronics smartphone apple high
750544,1004768_1004767,1,1004768,1004767,253.36,electronics smartphone samsung medium,view,1004767,2053013555631882655,electronics smartphone,samsung,254.82,555448677,3886fcc8-3618-4bb0-a9f4-614bfb2d5263,179.26,725.08,medium,electronics smartphone samsung medium
771972,1004781_1004565,1,1004781,1004565,278.31,electronics smartphone huawei medium,view,1004565,2053013555631882655,electronics smartphone,huawei,177.47,534620200,65839078-ffd5-4450-9f5b-a93da2acc5d0,179.26,725.08,low,electronics smartphone huawei low


# Final recommendation

In [31]:
df_final = df_final[['combo', 'counts', 'product_1', 'product_2', 'price_1', 'price_2', 'metadata_1', 'metadata_2']]

In [32]:
df_final

,combo,counts,product_1,product_2,price_1,price_2,metadata_1,metadata_2
0,1005115_1005105,2,1005115,1005105,975.57,1415.48,electronics smartphone apple high,electronics smartphone apple high
10512,1004767_1004836,2,1004767,1004836,254.82,241.18,electronics smartphone samsung medium,electronics smartphone samsung medium
18575,5100722_1004767,1,5100722,1004767,169.63,254.82,electronics clocks huawei medium,electronics smartphone samsung medium
40003,1004258_1002544,1,1004258,1002544,735.05,464.13,electronics smartphone apple high,electronics smartphone apple medium
49982,1004809_1004258,1,1004809,1004258,125.58,735.05,electronics smartphone xiaomi low,electronics smartphone apple high
...,...,...,...,...,...,...,...,...
743425,1005104_1003310,1,1005104,1003310,975.57,747.23,electronics smartphone apple high,electronics smartphone apple high
746146,1005115_1004258,1,1005115,1004258,975.57,735.05,electronics smartphone apple high,electronics smartphone apple high
750544,1004768_1004767,1,1004768,1004767,253.36,254.82,electronics smartphone samsung medium,electronics smartphone samsung medium
771972,1004781_1004565,1,1004781,1004565,278.31,177.47,electronics smartphone huawei medium,electronics smartphone huawei low


In [33]:
df_final = df_final[df_final['metadata_1'] != df_final['metadata_2']]

In [35]:
df_final.head()

,combo,counts,product_1,product_2,price_1,price_2,metadata_1,metadata_2
18575,5100722_1004767,1,5100722,1004767,169.63,254.82,electronics clocks huawei medium,electronics smartphone samsung medium
40003,1004258_1002544,1,1004258,1002544,735.05,464.13,electronics smartphone apple high,electronics smartphone apple medium
49982,1004809_1004258,1,1004809,1004258,125.58,735.05,electronics smartphone xiaomi low,electronics smartphone apple high
54380,1005160_11300053,1,1005160,11300053,231.41,17.25,electronics smartphone xiaomi medium,electronics telephone texet low
54401,1003317_1002524,1,1003317,1002524,957.53,515.67,electronics smartphone apple high,electronics smartphone apple medium


In [36]:
df_final.head()

,combo,counts,product_1,product_2,price_1,price_2,metadata_1,metadata_2
18575,5100722_1004767,1,5100722,1004767,169.63,254.82,electronics clocks huawei medium,electronics smartphone samsung medium
40003,1004258_1002544,1,1004258,1002544,735.05,464.13,electronics smartphone apple high,electronics smartphone apple medium
49982,1004809_1004258,1,1004809,1004258,125.58,735.05,electronics smartphone xiaomi low,electronics smartphone apple high
54380,1005160_11300053,1,1005160,11300053,231.41,17.25,electronics smartphone xiaomi medium,electronics telephone texet low
54401,1003317_1002524,1,1003317,1002524,957.53,515.67,electronics smartphone apple high,electronics smartphone apple medium


In [39]:
df_final["1.1"] = df_final["metadata_1"].str.split(" ",expand=True)[0].astype('category')
df_final["1.2"] = df_final["metadata_1"].str.split(" ",expand=True)[1].astype('category')
df_final.head()

/tmp/ipykernel_20101/874055066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["1.1"] = df_final["metadata_1"].str.split(" ",expand=True)[0].astype('category')
/tmp/ipykernel_20101/874055066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["1.2"] = df_final["metadata_1"].str.split(" ",expand=True)[1].astype('category')


,combo,counts,product_1,product_2,price_1,price_2,metadata_1,metadata_2,1.1,1.2
18575,5100722_1004767,1,5100722,1004767,169.63,254.82,electronics clocks huawei medium,electronics smartphone samsung medium,electronics,clocks
40003,1004258_1002544,1,1004258,1002544,735.05,464.13,electronics smartphone apple high,electronics smartphone apple medium,electronics,smartphone
49982,1004809_1004258,1,1004809,1004258,125.58,735.05,electronics smartphone xiaomi low,electronics smartphone apple high,electronics,smartphone
54380,1005160_11300053,1,1005160,11300053,231.41,17.25,electronics smartphone xiaomi medium,electronics telephone texet low,electronics,smartphone
54401,1003317_1002524,1,1003317,1002524,957.53,515.67,electronics smartphone apple high,electronics smartphone apple medium,electronics,smartphone


In [48]:
df_final["2.1"] = df_final["metadata_2"].str.split(" ",expand=True)[0].astype('category')
df_final["2.2"] = df_final["metadata_2"].str.split(" ",expand=True)[1].astype('category')
df_final.shape

/tmp/ipykernel_20101/2188218553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["2.1"] = df_final["metadata_2"].str.split(" ",expand=True)[0].astype('category')
/tmp/ipykernel_20101/2188218553.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["2.2"] = df_final["metadata_2"].str.split(" ",expand=True)[1].astype('category')


(87, 14)

In [43]:
df_final['test_1'] = df_final[['1.1', '1.2']].apply(lambda x: ' '.join(x), axis = 1)
df_final['test_2'] = df_final[['2.1', '2.2']].apply(lambda x: ' '.join(x), axis = 1)

/tmp/ipykernel_20101/1810607773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['test_1'] = df_final[['1.1', '1.2']].apply(lambda x: ' '.join(x), axis = 1)
/tmp/ipykernel_20101/1810607773.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['test_2'] = df_final[['2.1', '2.2']].apply(lambda x: ' '.join(x), axis = 1)


In [44]:
df_final.head()

,combo,counts,product_1,product_2,price_1,price_2,metadata_1,metadata_2,1.1,1.2,2.1,2.2,test_1,test_2
18575,5100722_1004767,1,5100722,1004767,169.63,254.82,electronics clocks huawei medium,electronics smartphone samsung medium,electronics,clocks,electronics,smartphone,electronics clocks,electronics smartphone
40003,1004258_1002544,1,1004258,1002544,735.05,464.13,electronics smartphone apple high,electronics smartphone apple medium,electronics,smartphone,electronics,smartphone,electronics smartphone,electronics smartphone
49982,1004809_1004258,1,1004809,1004258,125.58,735.05,electronics smartphone xiaomi low,electronics smartphone apple high,electronics,smartphone,electronics,smartphone,electronics smartphone,electronics smartphone
54380,1005160_11300053,1,1005160,11300053,231.41,17.25,electronics smartphone xiaomi medium,electronics telephone texet low,electronics,smartphone,electronics,telephone,electronics smartphone,electronics telephone
54401,1003317_1002524,1,1003317,1002524,957.53,515.67,electronics smartphone apple high,electronics smartphone apple medium,electronics,smartphone,electronics,smartphone,electronics smartphone,electronics smartphone


In [45]:
df_final_2 = df_final[df_final['test_1'] != df_final['test_2']]

In [49]:
df_final_2

,combo,counts,product_1,product_2,price_1,price_2,metadata_1,metadata_2,1.1,1.2,2.1,2.2,test_1,test_2
18575,5100722_1004767,1,5100722,1004767,169.63,254.82,electronics clocks huawei medium,electronics smartphone samsung medium,electronics,clocks,electronics,smartphone,electronics clocks,electronics smartphone
54380,1005160_11300053,1,1005160,11300053,231.41,17.25,electronics smartphone xiaomi medium,electronics telephone texet low,electronics,smartphone,electronics,telephone,electronics smartphone,electronics telephone
88838,21404513_1004856,1,21404513,1004856,16.73,130.76,electronics clocks casio low,electronics smartphone samsung low,electronics,clocks,electronics,smartphone,electronics clocks,electronics smartphone
145188,5100562_1003316,1,5100562,1003316,296.62,946.97,electronics clocks apple medium,electronics smartphone apple high,electronics,clocks,electronics,smartphone,electronics clocks,electronics smartphone
185811,4803977_21402211,1,4803977,21402211,107.80,69.76,electronics audio headphone samsung medium,electronics clocks lorus low,electronics,audio,electronics,clocks,electronics audio,electronics clocks
234915,1005210_32200007,1,1005210,32200007,59.18,268.02,electronics smartphone jinga low,kids swing redford high,electronics,smartphone,kids,swing,electronics smartphone,kids swing
234931,4804295_1004856,1,4804295,1004856,23.13,130.76,electronics audio headphone xiaomi low,electronics smartphone samsung low,electronics,audio,electronics,smartphone,electronics audio,electronics smartphone
297552,1004886_8801002,1,1004886,8801002,153.04,36.01,electronics smartphone oppo low,electronics telephone nokia medium,electronics,smartphone,electronics,telephone,electronics smartphone,electronics telephone
298939,5100855_1005115,1,5100855,1005115,617.52,975.57,electronics clocks apple high,electronics smartphone apple high,electronics,clocks,electronics,smartphone,electronics clocks,electronics smartphone
363017,1004258_1201292,1,1004258,1201292,735.05,411.57,electronics smartphone apple high,electronics tablet apple medium,electronics,smartphone,electronics,tablet,electronics smartphone,electronics tablet
